In [2]:
import sagemaker
import boto3
from time import gmtime, strftime, sleep

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
!pip install -U awswrangler pyqt5 pyqtwebengine bokeh ipython pylint
!pip install sagemaker-data-insights==0.4.0
import awswrangler as wr


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd

In [35]:
def train_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 7)

def val_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 12) and (int(partitions["month"]) > 7)


In [36]:
path = "s3://{}/meterdataset.parquet/".format(bucket)
df_parquet_train = wr.s3.read_parquet(
    path, 
    #columns=["artists", "track_name", "popularity"], 
    partition_filter=train_filter, 
    dataset=True
)
df_parquet_train.shape

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


(5112, 7)

In [37]:
path = "s3://{}/meterdataset.parquet/".format(bucket)
df_parquet_val = wr.s3.read_parquet(
    path, 
    #columns=["artists", "track_name", "popularity"], 
    partition_filter=val_filter, 
    dataset=True
)
df_parquet_val.shape

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


(3672, 7)

In [38]:
df_parquet_train = df_parquet_train.drop(columns=['year','month'])
df_parquet_train['weekday'] = df_parquet_train['weekday'].astype(float)

In [39]:
df_parquet_train

,kWh,weekday,hour,temperature,Holiday
timestamp,,,,,
2012-01-01 00:00:00,0.230723,6.0,0.0,12.12,-1.0
2012-01-01 01:00:00,0.268496,6.0,1.0,12.59,-1.0
2012-01-01 02:00:00,0.228618,6.0,2.0,12.45,-1.0
2012-01-01 03:00:00,0.191603,6.0,3.0,12.03,-1.0
2012-01-01 04:00:00,0.144564,6.0,4.0,12.04,-1.0
...,...,...,...,...,...
2012-07-31 19:00:00,0.236679,1.0,19.0,18.52,-1.0
2012-07-31 20:00:00,0.241126,1.0,20.0,17.48,-1.0
2012-07-31 21:00:00,0.253411,1.0,21.0,16.85,-1.0


In [40]:
df_parquet_val = df_parquet_val.drop(columns=['year','month'])
df_parquet_val['weekday'] = df_parquet_val['weekday'].astype(float)

In [41]:
df_parquet_val.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3672 entries, 2012-10-01 00:00:00 to 2012-09-30 23:00:00
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   kWh          3672 non-null   float64
 1   weekday      3672 non-null   float64
 2   hour         3672 non-null   float64
 3   temperature  3672 non-null   float64
 4   Holiday      3672 non-null   float64
dtypes: float64(5)
memory usage: 172.1 KB


In [42]:
train_string = "housepow_train.parquet"
val_string = "housepow_val.parquet"
df_parquet_train.to_parquet(train_string)
df_parquet_val.to_parquet(val_string)

In [43]:
prefix = "Smart-Grid-prediction-lightGBM"

In [44]:
from sagemaker import image_uris, model_uris, script_uris

In [47]:
%%time
sess.upload_data(
    train_string, bucket=bucket, key_prefix=prefix + "/" + "training1"
)

sess.upload_data(
    val_string, bucket=bucket, key_prefix=prefix + "/" + "validation1"
)

In [48]:
train_model_id, train_model_version, train_scope = "lightgbm-regression-model", "2.1.0", "training"
training_instance_type = "ml.m5.xlarge"

# Retrieve the docker image
training_image = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

In [49]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters[
    "num_boost_round"
] = "500"  # The same hyperparameter is named as "iterations" for CatBoost
print(hyperparameters)

{'num_boost_round': '500', 'early_stopping_rounds': '30', 'metric': 'auto', 'learning_rate': '0.009', 'num_leaves': '67', 'feature_fraction': '0.74', 'bagging_fraction': '0.53', 'bagging_freq': '5', 'max_depth': '11', 'min_data_in_leaf': '26', 'max_delta_step': '0.0', 'lambda_l1': '0.0', 'lambda_l2': '0.0', 'boosting': 'gbdt', 'min_gain_to_split': '0.0', 'tree_learner': 'serial', 'feature_fraction_bynode': '1.0', 'is_unbalance': 'False', 'max_bin': '255', 'tweedie_variance_power': '1.5', 'num_threads': '0', 'verbosity': '1', 'use_dask': 'False'}


#### Construct a SageMaker generic estimator using the SageMaker LGBMcontainer

In [50]:
dataset_uri_prefix = "s3://" + bucket + "/" + prefix + "/training1/"
dataset_uri_prefix_val = "s3://" + bucket + "/" + prefix + "/validation1/"
training_output_path = "s3://" + bucket + "/" + prefix + "/training_output"

#### Set hyperparameters

In [58]:
from sagemaker.tuner import ContinuousParameter, IntegerParameter, HyperparameterTuner

use_amt = True

In [52]:
if train_model_id == "lightgbm-regression-model":
    hyperparameter_ranges = {
        "learning_rate": ContinuousParameter(1e-4, 1, scaling_type="Logarithmic"),
        "num_boost_round": IntegerParameter(2, 30),
        "early_stopping_rounds": IntegerParameter(2, 30),
        "num_leaves": IntegerParameter(10, 50),
        "feature_fraction": ContinuousParameter(0, 1),
        "bagging_fraction": ContinuousParameter(0, 1),
        "bagging_freq": IntegerParameter(1, 10),
        "max_depth": IntegerParameter(5, 30),
        "min_data_in_leaf": IntegerParameter(5, 50),
    }

In [56]:
from sagemaker.estimator import Estimator

tabular_estimator = Estimator(
    role=role,
    image_uri=training_image,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=training_output_path,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [54]:
train_data = sagemaker.inputs.TrainingInput(
    dataset_uri_prefix,
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

validation_data = sagemaker.inputs.TrainingInput(
    dataset_uri_prefix_val,
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, 'validation': validation_data}

In [59]:
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"SmartGrid-{train_model_id}-training")

if use_amt:
    tuner = HyperparameterTuner(
        tabular_estimator,
        "rmse",
        hyperparameter_ranges,
        [{"Name": "rmse", "Regex": "rmse: ([0-9\\.]+)"}],
        max_jobs=10,
        max_parallel_jobs=2,
        objective_type="Minimize",
        base_tuning_job_name=training_job_name,
    )

    tuner.fit(inputs=data_channels, logs=True)

else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    tabular_estimator.fit(
        inputs=data_channels, logs=True, job_name=training_job_name
    )

INFO:sagemaker:Creating hyperparameter tuning job with name: SmartGrid-lightgbm-r-240225-0244


......................................................................!


In [71]:
sagemaker_obj = boto3.client("sagemaker")
model_name = tuner.best_training_job()
print(model_name)


info = sagemaker_obj.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": training_image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker_obj.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

SmartGrid-lightgbm-r-240225-0244-009-79f1eb9c
arn:aws:sagemaker:us-east-1:645257835735:model/smartgrid-lightgbm-r-240225-0244-009-79f1eb9c


In [46]:
#predictor.delete_endpoint()

In [ ]:
info

In [87]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_name = name_from_base(f"SmartGrid-{train_model_id}-EP-")
                            
inference_instance_type = "ml.m5.large"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"SmartGrid-{train_model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
predictor = (tuner if use_amt else tabular_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

#print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


2024-02-25 02:50:09 Starting - Found matching resource for reuse
2024-02-25 02:50:09 Downloading - Downloading the training image
2024-02-25 02:50:09 Training - Training image download completed. Training in progress.
2024-02-25 02:50:09 Uploading - Uploading generated training model
2024-02-25 02:50:09 Completed - Resource released due to keep alive period expiry


INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-645257835735/Smart-Grid-prediction-lightGBM/training_output/SmartGrid-lightgbm-r-240225-0244-009-79f1eb9c/output/model.tar.gz), script artifact (s3://jumpstart-cache-prod-us-east-1/source-directory-tarballs/lightgbm/inference/regression/v1.2.1/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-645257835735/sagemaker-jumpstart-2024-02-25-03-46-40-346/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2024-02-25-03-46-40-346
INFO:sagemaker:Creating endpoint-config with name SmartGrid-lightgbm-regression-model--2024-02-25-03-46-35-075
INFO:sagemaker:Creating endpoint with name SmartGrid-lightgbm-regression-model--2024-02-25-03-46-35-075


-----!Created EndpointConfig: arn:aws:sagemaker:us-east-1:645257835735:endpoint-config/smartgrid-lightgbm-regression-model-2024-02-25-03-14-56


In [80]:
# Deploy our model to real-time endpoint

endpoint_name = f"SmartGrid-{train_model_id}-EP-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker_obj.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [83]:
# Wait for endpoint to spin up
from time import sleep
sagemaker_obj.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker_obj.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...


KeyboardInterrupt: 

In [84]:
res = sagemaker_obj.describe_endpoint(EndpointName=endpoint_name)
res["EndpointStatus"]

'Creating'

In [5]:
sagemaker_obj = boto3.client("sagemaker")
sagemaker_obj.delete_endpoint(EndpointName=endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "smartgrid-lightgbm-regression-model-2024-02-25-03-14-56".

In [3]:
endpoint_name = 'smartgrid-lightgbm-regression-model-2024-02-25-03-14-56'